In [10]:
import numpy as np
from tensorflow.keras.datasets import mnist
from cvxopt import matrix, solvers
import time

In [11]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

digits_of_interest = [2, 3, 8, 9]
mask = np.isin(y_train, digits_of_interest)
x_train, y_train = x_train[mask], y_train[mask]
mask = np.isin(y_test, digits_of_interest)
x_test, y_test = x_test[mask], y_test[mask]

x_train, y_train = x_train[:5000], y_train[:5000]
x_test, y_test = x_test[:1250], y_test[:1250]

x_train = x_train.reshape(-1, 784) / 255.0
x_test = x_test.reshape(-1, 784) / 255.0

In [12]:
# Ensure labels are -1 and 1
y_train = np.where(y_train == 2, 1, -1)
y_test = np.where(y_test == 2, 1, -1)

In [13]:
def rbf_kernel(x1, x2, gamma=0.01):
    if x1.ndim == 1:
        x1 = x1.reshape(1, -1)
    if x2.ndim == 1:
        x2 = x2.reshape(1, -1)
    squared_distances = np.sum((x1[:, np.newaxis, :] - x2[np.newaxis, :, :]) ** 2, axis=2)
    return np.exp(-gamma * squared_distances)

In [14]:
def compute_kernel_matrix(X, kernel_function):
    n_samples = X.shape[0]
    K = np.zeros((n_samples, n_samples))
    for i in range(n_samples):
        K[i, :] = kernel_function(X[i], X)
    return K

In [15]:
def prepare_matrices(X, y, C):
    y = y.astype(float)
    K = compute_kernel_matrix(X, lambda x1, x2: rbf_kernel(x1, x2))
    P = matrix(np.outer(y, y) * K)
    q = matrix(-np.ones((X.shape[0], 1)))
    G = matrix(np.vstack((-np.eye(X.shape[0]), np.eye(X.shape[0]))))
    h = matrix(np.hstack((np.zeros(X.shape[0]), np.ones(X.shape[0]) * C)))
    A = matrix(y.reshape(1, -1))
    b = matrix(np.zeros(1))
    return P, q, G, h, A, b

In [16]:
C = 0.1
P, q, G, h, A, b = prepare_matrices(x_train, y_train, C)

# Start timing the training process
start_time = time.time()

sol = solvers.qp(P, q, G, h, A, b)

# End timing the training process
end_time = time.time()
training_time = end_time - start_time

alphas = np.array(sol['x']).flatten()

     pcost       dcost       gap    pres   dres
 0: -2.7445e+02 -1.0471e+03  4e+04  3e+01  4e-15
 1: -1.0429e+02 -9.5346e+02  3e+03  1e+00  4e-15
 2: -7.0878e+01 -4.6921e+02  5e+02  1e-01  2e-15
 3: -7.5749e+01 -1.5682e+02  9e+01  2e-02  3e-15
 4: -8.5765e+01 -1.2059e+02  4e+01  8e-03  2e-15
 5: -9.0639e+01 -1.0796e+02  2e+01  3e-03  2e-15
 6: -9.3154e+01 -1.0220e+02  9e+00  1e-03  1e-15
 7: -9.4850e+01 -9.8735e+01  4e+00  5e-04  2e-15
 8: -9.5866e+01 -9.6843e+01  1e+00  7e-05  2e-15
 9: -9.6215e+01 -9.6317e+01  1e-01  3e-06  2e-15
10: -9.6259e+01 -9.6264e+01  5e-03  7e-08  2e-15
11: -9.6262e+01 -9.6262e+01  2e-04  2e-09  2e-15
12: -9.6262e+01 -9.6262e+01  3e-06  4e-11  2e-15
Optimal solution found.


In [17]:
# Debugging: Print the alphas
print("Alphas:", alphas)

support_vector_indices = np.where(alphas > 1e-5)[0]
support_vectors = x_train[support_vector_indices]
support_vector_labels = y_train[support_vector_indices]
support_vector_alphas = alphas[support_vector_indices]

# Debugging: Print the support vector details
print("Support Vector Indices:", support_vector_indices)
print("Support Vector Alphas:", support_vector_alphas)


def compute_intercept(support_vectors, support_vector_labels, support_vector_alphas, kernel_function):
    intercept = 0
    for i in range(len(support_vector_alphas)):
        intercept += support_vector_labels[i]
        intercept -= np.sum(support_vector_alphas * support_vector_labels * kernel_function(support_vectors, support_vectors[i]))
    if len(support_vector_alphas) > 0:
        intercept /= len(support_vector_alphas)
    else:
        intercept = 0
    return intercept

intercept = compute_intercept(support_vectors, support_vector_labels, support_vector_alphas, lambda x1, x2: rbf_kernel(x1, x2))

Alphas: [4.29798454e-10 9.99999991e-02 4.67116719e-10 ... 3.84375595e-10
 3.03997595e-10 3.48081994e-10]
Support Vector Indices: [   1    5    8 ... 4988 4989 4990]
Support Vector Alphas: [0.1        0.1        0.01927898 ... 0.1        0.1        0.1       ]


In [18]:
def predict(X, support_vectors, support_vector_labels, support_vector_alphas, intercept, kernel_function):
    predictions = np.zeros(X.shape[0])
    for i in range(X.shape[0]):
        prediction = 0
        for alpha, sv_y, sv in zip(support_vector_alphas, support_vector_labels, support_vectors):
            prediction += alpha * sv_y * kernel_function(X[i], sv)
        predictions[i] = prediction
    return np.sign(predictions + intercept)

y_train_pred = predict(x_train, support_vectors, support_vector_labels, support_vector_alphas, intercept, lambda x1, x2: rbf_kernel(x1, x2))
y_test_pred = predict(x_test, support_vectors, support_vector_labels, support_vector_alphas, intercept, lambda x1, x2: rbf_kernel(x1, x2))

train_accuracy = np.mean(y_train == y_train_pred)
test_accuracy = np.mean(y_test == y_test_pred)

print(f"Training accuracy: {train_accuracy:.2f}")
print(f"Test accuracy: {test_accuracy:.2f}")
# Print the training time
print(f"Training time: {training_time:.2f} seconds")

/var/folders/0y/rkt1cks911v4smbrjw1q78v80000gn/T/ipykernel_64416/1375560065.py:8: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predictions[i] = prediction


Training accuracy: 0.97
Test accuracy: 0.96
Training time: 28.08 seconds
